# 텍스트 분류 1차 시도 (O,X만 모아서 8529행)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df2 = pd.read_csv('train_text_data4.csv', sep='\t', encoding='UTF-8')

In [4]:
df2.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN  
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN  
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN  
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN  
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN

In [5]:
df2.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인'],
      dtype='object')

In [6]:
df2.shape

(8529, 16)

In [7]:
category_count = df2 ['대상 선정'].value_counts()
category_count

X    5211
O    3318
Name: 대상 선정, dtype: int64

In [8]:
category_count = df2 ['책임구'].value_counts()
category_count

구매      2564
생산공장     722
구매        22
연구소        7
 구매        1
Name: 책임구, dtype: int64

In [9]:
category_count = df2 ['원인'].value_counts()
category_count

부품       2586
조립불량      270
결선불량      201
체결불량       99
기타         87
미결선        36
이종/간섭      17
이물          8
설계/구조       7
액주입불량       4
Name: 원인, dtype: int64

# ◈ Null 제거

In [10]:
df_bak = df2.copy()

In [11]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df2 = df2.fillna('N')

In [12]:
df2.shape

(8529, 16)

In [13]:
category_count = df2 ['대상 선정'].value_counts()
category_count

X    5211
O    3318
Name: 대상 선정, dtype: int64

In [14]:
category_count = df2 ['책임구'].value_counts()
category_count

N       5213
구매      2564
생산공장     722
구매        22
연구소        7
 구매        1
Name: 책임구, dtype: int64

In [15]:
category_count = df2 ['원인'].value_counts()
category_count

N        5214
부품       2586
조립불량      270
결선불량      201
체결불량       99
기타         87
미결선        36
이종/간섭      17
이물          8
설계/구조       7
액주입불량       4
Name: 원인, dtype: int64

In [16]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
# df2['caus_pred'] = df2['원인'].astype(str).copy()
df2 ['target_predict'] = df2 ['대상 선정'].astype(str).copy()
# df2 ['train_charge_predict'] = df2 ['책임구'].astype(str).copy()
# df2 ['train_cause_predict'] = df2 ['원인'].astype(str).copy()

In [17]:
def preprocessing(text):
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   #-> 특수문자 제거
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)         #-> 한글, 영문, 숫자만 남기고 모두 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)              #-> 한글, 영문만 남기고 모두 제거
    return text

In [18]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [19]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 109 ms


In [20]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 46.8 ms


# ◈ Train Data & Test Data 만들기

In [21]:
df2 = df2.reindex()

In [22]:
df2.shape

(8529, 17)

In [23]:
split_count = int(df2.shape[0]*0.9)
split_count

7676

In [24]:
df2_train = df2[:split_count].copy()
df2_train.shape

(7676, 17)

In [25]:
df2_test = df2[split_count:].copy()
df2_test.shape

(853, 17)

# ◈ 단어 벡터화

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word',    # 캐릭터 단위로도 벡터화가능
                             tokenizer = None,     # 토크나이저를 따로 지정해 줄 수도 있음
                             preprocessor = None,  # 전처리 도구
                             stop_words = stops,   # 불용어 nltk등의 도구를 사용할 수도 있음
                             min_df = 1,           # 토큰출현 최소문서개수:오타 및 자주 출현 안하는 특수전문용어 제거에 좋음
                             ngram_range=(1, 3),   # BOW단위를 1~3개로 지정
                             max_features = 20000  # 만들 피처수, 단어수가 됨
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [27]:
%%time
train_feature_vector = vectorizer.fit_transform(df2_train['clm_prtl_sbc'])
train_feature_vector.shape

Wall time: 1.15 s


In [28]:
%%time
test_feature_vector = vectorizer.fit_transform(df2_test['clm_prtl_sbc'])
test_feature_vector.shape

Wall time: 156 ms


In [29]:
vocab1 = vectorizer.get_feature_names()
print(len(vocab1))
vocab1[:10]

20000


['ab',
 'assy',
 'assy 교환',
 'cm',
 'cs',
 'cs 차량으로',
 'db',
 'db로',
 'db에서',
 'db에서 db로']

In [30]:
dist = np.sum(train_feature_vector, axis=0)
vec_df2 = pd.DataFrame(dist, columns = vocab1)
vec_df2

ab  assy  assy 교환  cm  cs  cs 차량으로  db  db로  db에서  db에서 db로 ...  흡음제  \
0  39     8        3   4   3        6   3    4     3         9 ...    3   

   흡음제 작업후  흡음제 작업후 재조립후  흡입구에  흡입구에 방음지  흡입구에 방음지 부착  흡입호스  흡입호스 재조립  \
0        5            16     3         4            9     3         5   

   흡입호스 재조립 개선  히터  
0           23   3  

[1 rows x 20000 columns]

In [31]:
vec_df2.columns

Index(['ab', 'assy', 'assy 교환', 'cm', 'cs', 'cs 차량으로', 'db', 'db로', 'db에서',
       'db에서 db로',
       ...
       '흡음제', '흡음제 작업후', '흡음제 작업후 재조립후', '흡입구에', '흡입구에 방음지', '흡입구에 방음지 부착',
       '흡입호스', '흡입호스 재조립', '흡입호스 재조립 개선', '히터'],
      dtype='object', length=20000)

# ◈ tf-idf 가중치 적용하기

In [32]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer (smooth_idf = False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [33]:
%%time
train_feature_tfidf = transformer.fit_transform (train_feature_vector)
train_feature_tfidf.shape

Wall time: 93.6 ms


In [34]:
%%time
test_feature_tfidf = transformer.fit_transform (test_feature_vector)
test_feature_tfidf.shape

Wall time: 78 ms


# ◈ Machine Learning

In [35]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier (
    n_estimators = 100, n_jobs = -1, random_state = 2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [36]:
y_label = df2_train['target_predict']
%time forest = forest.fit (train_feature_tfidf, y_label)

Wall time: 2.12 s


# ◈ Model 평가

In [37]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold (n_splits = 5, shuffle = True, random_state = 0)

scoring = 'accuracy'
%time score = cross_val_score (forest, train_feature_vector, y_label, cv = k_fold, n_jobs = -1, scoring = scoring)
score

Wall time: 33.8 s


array([0.73958333, 0.75700326, 0.76026059, 0.72052117, 0.75895765])

In [38]:
round (np.mean(score)*100,2)

74.73

# ◈ 예측 (Test Data 이용)

In [39]:
y_predict = forest.predict (test_feature_vector)
y_predict[:3]

array(['X', 'X', 'X'], dtype=object)

In [40]:
y_predict.shape

(853,)

In [41]:
output = pd.DataFrame (data = {'test_category_predict':y_predict})
output.head()

test_category_predict
0                     X
1                     X
2                     X
3                     O
4                     O

In [42]:
df2_test ['test_category_predict'] = y_predict

In [43]:
df2_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
7676               W  2018     20180912         ZZ5         Q57   소음/이음_작동시   
7677               W  2016     20160304         ZZ3         Q57   소음/이음_작동시   
7678               W  2018     20180905         ZZ7         Q57   소음/이음_작동시   
7679               W  2015     20150710         ZZ5         Q57   소음/이음_작동시   
7680               W  2017     20170418         ZZ4         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
7676         KV11          YP     A9          TT        83940   
7677         KV11          YP     A9          TT        82540   
7678         KV11          YP     A9          TT        846D0   
7679         KV11          YP     A9          TT        87620   
7680         KV11          YP     A9          TT        87620   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  \
7676   C   슬라이딩 도어 우측 열림시 찍 찌직하는 이음 발생점검  센타 롤러에서 이음...     X  N   N  N   
7677                      C  RH 런고무 불량으로 윈도우 작동시 끽끼기긱소음     X  N   N  N   
7678   C  무선 충전 부분을 손으로 아래 방향으로 누르면 유격 있으면서 소리가남어퍼 콘...     X  N   N  N   
7679   C  현상사이드 미러 작동시 우측사이드미러 이음굉음소리가 남으로 인하여 입고함조치...     X  N   N  N   
7680   C  현상우측 사이드 미러 접힐시 끼이익 끼이익 소음 발생 점검내용및원인 사이드 ...     X  N   N  N   

     target_predict test_category_predict  
7676              X                     X  
7677              X                     X  
7678              X                     X  
7679              X                     O  
7680              X                     O

# ◈ 결과검증

In [44]:
df2_test ['predict_diff'] = 0
df2_test ['predict_diff'] = (df2_test['대상 선정'] == df2_test['test_category_predict']) == 1
df2_test['predict_diff'] = df2_test['predict_diff'].astype(int)
df2_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
7676               W  2018     20180912         ZZ5         Q57   소음/이음_작동시   
7677               W  2016     20160304         ZZ3         Q57   소음/이음_작동시   
7678               W  2018     20180905         ZZ7         Q57   소음/이음_작동시   
7679               W  2015     20150710         ZZ5         Q57   소음/이음_작동시   
7680               W  2017     20170418         ZZ4         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
7676         KV11          YP     A9          TT        83940   
7677         KV11          YP     A9          TT        82540   
7678         KV11          YP     A9          TT        846D0   
7679         KV11          YP     A9          TT        87620   
7680         KV11          YP     A9          TT        87620   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  \
7676   C   슬라이딩 도어 우측 열림시 찍 찌직하는 이음 발생점검  센타 롤러에서 이음...     X  N   N  N   
7677                      C  RH 런고무 불량으로 윈도우 작동시 끽끼기긱소음     X  N   N  N   
7678   C  무선 충전 부분을 손으로 아래 방향으로 누르면 유격 있으면서 소리가남어퍼 콘...     X  N   N  N   
7679   C  현상사이드 미러 작동시 우측사이드미러 이음굉음소리가 남으로 인하여 입고함조치...     X  N   N  N   
7680   C  현상우측 사이드 미러 접힐시 끼이익 끼이익 소음 발생 점검내용및원인 사이드 ...     X  N   N  N   

     target_predict test_category_predict  predict_diff  
7676              X                     X             1  
7677              X                     X             1  
7678              X                     X             1  
7679              X                     O             0  
7680              X                     O             0

In [45]:
diff_sum = df2_test['predict_diff'].sum()
diff_sum

462

In [46]:
y_predict_len = len(y_predict)
y_predict_len

853

# ◈ 정확도

In [47]:
Acy = round(diff_sum/y_predict_len*100,1)
Acy

54.2